In [6]:
import numpy as np
import sys
import os

MINM = -1000000000

#Hyperparameters
DELTA = 0.001
GAMMA = 0.999         
NOISE = 0  

#Define all pos
all_pos=[]
all_pos.append('W')
all_pos.append('N')
all_pos.append('E')
all_pos.append('S')
all_pos.append('C')

#Define all state
all_states=[]
all_states.append('D')
all_states.append('R')

max_mat = 2
max_arrow = 3
E_left = 0

#Reward for each action
action_reward = {
    'UP': -10,
    'LEFT': -10,
    'DOWN': -10,
    'RIGHT': -10,
    'STAY': -10,
    'SHOOT': -10,
    'HIT': -10,
    'CRAFT': -10,
    'GATHER': -10,
    'NONE': -10
}

#Define rewards for all states
rewards = {}
for s in all_pos:
    rewards[s] = 0

#Dictionnary of possible actions. We have two "end" states (1,2 and 2,2)
actions = {
    'W':('RIGHT', 'STAY', 'SHOOT', 'NONE'), 
    'N':('DOWN', 'STAY', 'CRAFT', 'NONE'),
    'E':('LEFT', 'STAY', 'SHOOT', 'HIT', 'NONE'),
    'S':('UP', 'STAY', 'GATHER', 'NONE'),
    'C':('UP', 'DOWN', 'LEFT', 'RIGHT', 'STAY', 'SHOOT', 'HIT', 'NONE')
    }

#Dictionary for integer mapping of pos of IJ
pos_map = {
    'W' : 0,
    'N' : 1,
    'E' : 2,
    'S' : 3,
    'C' : 4
}

#Dictionary for integer mapping of states of MM
state_map = {
    'D' : 0,
    'R' : 1
}

#Dictionary for integer mapping of actions
action_map = {
    'UP': 0,
    'LEFT': 1,
    'DOWN': 2,
    'RIGHT': 3,
    'STAY': 4,
    'SHOOT': 5,
    'HIT': 6,
    'CRAFT': 7,
    'GATHER': 8,
    'NONE': 9
}

#Define an initial policy
policy={}
for s in actions.keys():
    policy[s] = np.random.choice(actions[s])
print(policy)

#Define success probabilities for states
probability = {}
for s in all_pos:
    if s == 'E' or s == 'W':
        probability[s] = 1
    else:
        probability[s] = 0.85
print(probability)
        
#Define fail action for states
fail = {}
for s in all_pos:
    if s == 'E' or s == 'W':
        fail[s] = s
    else:
        fail[s] = 'E'
print(fail)

{'W': 'NONE', 'N': 'DOWN', 'E': 'LEFT', 'S': 'UP', 'C': 'SHOOT'}
{'W': 1, 'N': 0.85, 'E': 1, 'S': 0.85, 'C': 0.85}
{'W': 'W', 'N': 'E', 'E': 'E', 'S': 'E', 'C': 'E'}


In [27]:
def prob(a,p,m,arr,s,h,p1,m1,arr1,s1,h1):
    if a == 'UP':
        if m!=m1 or arr!=arr1 or s!=s1 or h!=h1:
            return 0
        if p == 'S':
            if p1 == 'C':
                return probability[p]
            elif p1 == fail[p]:
                return 1-probability[p]
            else:
                return 0
        elif p == 'C':
            if p1 == 'N':
                return probability[p]
            elif p1 == fail[p]:
                return 1-probability[p]
            else:
                return 0
        else:
            return 0
    elif a == 'DOWN':
        if m!=m1 or arr!=arr1 or s!=s1 or h!=h1:
            return 0
        if p == 'N':
            if p1 == 'C':
                return probability[p]
            elif p1 == fail[p]:
                return 1-probability[p]
            else:
                return 0
        elif p == 'C':
            if p1 == 'S':
                return probability[p]
            elif p1 == fail[p]:
                return 1-probability[p]
            else:
                return 0
        else:
            return 0
    elif a == 'LEFT':
        if m!=m1 or arr!=arr1 or s!=s1 or h!=h1:
            return 0
        if p == 'E':
            if p1 == 'C' or (p1 == 'W' and E_left == 1):
                return probability[p]
            elif p1 == fail[p]:
                return 1-probability[p]
            else:
                return 0
        elif p == 'C':
            if p1 == 'W':
                return probability[p]
            elif p1 == fail[p]:
                return 1-probability[p]
            else:
                return 0
        else:
            return 0
    elif a == 'RIGHT':
        if m!=m1 or arr!=arr1 or s!=s1 or h!=h1:
            return 0
        if p == 'W':
            if p1 == 'C':
                return probability[p]
            elif p1 == fail[p]:
                return 1-probability[p]
            else:
                return 0
        elif p == 'C':
            if p1 == 'E':
                return probability[p]
            elif p1 == fail[p]:
                return 1-probability[p]
            else:
                return 0
        else:
            return 0
    elif a == 'STAY':
        if m!=m1 or arr!=arr1 or s!=s1 or h!=h1:
            return 0
        if p1 == p:
            return probability[p]
        elif p1 == fail[p]:
            return 1-probability[p]
        else:
            return 0
        
    elif a == 'SHOOT':
        if p!=p1 or m!=m1 or s!=s1:
            return 0
        if arr == arr1+1 and (p == 'C' or p == 'E' or p == 'W'):
            if p == 'C':
                if h == h1+1:
                    return 0.5
                elif h == h1:
                    return 0.5
                else:
                    return 0
            elif p == 'E':
                if h == h1+1:
                    return 0.9
                elif h == h1:
                    return 0.1
                else:
                    return 0
            elif p == 'W':
                if h == h1+1:
                    return 0.25
                elif h == h1:
                    return 0.75
                else:
                    return 0
        else:
            return 0
        
    elif a == 'HIT':
        if p!=p1 or m!=m1 or s!=s1 or arr!=arr1:
            return 0
        if p == 'C':
            if h == h1+2:
                return 0.1
            elif h == h1:
                return 0.9
            else:
                return 0
        elif p == 'E':
            if h == h1+2:
                return 0.2
            elif h == h1:
                return 0.8
            else:
                return 0
        else:
            return 0
        
    elif a == 'CRAFT':
        if p!=p1 or s!=s1 or h!=h1:
            return 0
        if p == 'N' and m>=1 and m == m1+1:
            if arr+1 == arr1:
                return 0.5
            elif arr+2 == arr1:
                return 0.35
            elif arr+3 == arr1:
                return 0.15
            else:
                return 0
        else:
            return 0
        
    elif a == 'GATHER':
        if p!=p1 or s!=s1 or h!=h1 or arr!=arr1:
            return 0
        if p == 'S' and m<=1:
            if m+1 == m1:
                return 0.75
            elif m == m1:
                return 0.25
            else:
                return 0
        else:
            return 0
    
    elif a == 'NONE':
        return 0

def iterate():
    iteration = 0
    
    pos = 'W'
    mat = 0
    arrow = 0
    state = 'D'
    health = 4
    
    value = np.zeros((len(all_pos),max_mat,max_arrow,len(all_states),health))
    action = np.empty((len(all_pos),max_mat,max_arrow,len(all_states),health), dtype='<U7')
    
    while True:
        biggest_change = 0
        
        tmp = np.zeros((len(all_pos),max_mat,max_arrow,len(all_states),health))
        tmp[:] = MINM
        tmp[0,:,:] = 0
        
        for p in all_pos:
            for m in range(0,max_mat):
                for arr in range(0,max_arrow):
                    for s in all_states:
                        for h in range(1,health):

                            num_p = pos_map[p]
                            num_s = state_map[s]
                            # loop over actions
                            for a in actions[p]:
                                if a == 'SHOOT' and arr == 0:
                                    continue
                                elif a == 'CRAFT' and (m == 0 or arr == 3):
                                    continue
                                elif a == 'GATHER' and m == 2:
                                    continue
                                    
                                state_value = 0
                                for p1 in all_pos:
                                    for m1 in range(0,max_mat):
                                        for arr1 in range(0,max_arrow):
                                            for s1 in all_states:
                                                for h1 in range(0,health):
                                                    
                                                    reward = action_reward[a]
                                                    if h1 == 0:
                                                        reward += 50
                                                    elif s == 'R' and s1 == 'D' and (p == 'E' or p == 'C'):
                                                        reward -= 40
                                                    num_p1 = pos_map[p1] 
                                                    num_s1 = state_map[s1]
                                                    state_value += round((reward + GAMMA*value[num_p1,m1,arr1,num_s1,h1])*prob(a,p,m,arr,s,h,p1,m1,arr1,s1,h1),8)
                                
                                state_value = round(state_value,8)
                                # taking max of all the values
                                if state_value >= tmp[num_p,m,arr,num_s,h]:
                                    tmp[num_p,m,arr,num_s,h] = state_value
                                
                            biggest_change = max(biggest_change,abs(tmp[num_p,m,arr,num_s,h]-value[num_p,m,arr,num_s,h]))
                            
        np.copyto(value,tmp)
        
        tmpactn = np.zeros((len(all_pos),max_mat,max_arrow,len(all_states),health))
        action[:] = 'NONE'
        tmpactn[:] = MINM
        tmpactn[0,:,:] = -1        
                                
        for p in all_pos:
            for m in range(0,max_mat):
                for arr in range(0,max_arrow):
                    for s in all_states:
                        for h in range(1,health):

                            num_p = pos_map[p]
                            num_s = state_map[s]
                            # loop over actions
                            for a in actions[p]:
                                if a == 'SHOOT' and arr == 0:
                                    continue
                                elif a == 'CRAFT' and (m == 0 or arr == 3):
                                    continue
                                elif a == 'GATHER' and m == 2:
                                    continue
                                    
                                state_value = 0
                                for p1 in all_pos:
                                    for m1 in range(0,max_mat):
                                        for arr1 in range(0,max_arrow):
                                            for s1 in all_states:
                                                for h1 in range(0,health):
                                                    
                                                    reward = action_reward[a]
                                                    if h1 == 0:
                                                        reward += 50
                                                    elif s == 'R' and s1 == 'D' and (p == 'E' or p == 'C'):
                                                        reward -= 40
                                                    num_p1 = pos_map[p1] 
                                                    num_s1 = state_map[s1]
                                                        
                                                    state_value += round((reward + GAMMA*value[num_p1,m1,arr1,num_s1,h1])*prob(a,p,m,arr,s,h,p1,m1,arr1,s1,h1),8)
                                
                                state_value = round(state_value,8)
                                # Save the best of all actions for the state                          
                                if state_value >= tmpactn[num_p,m,arr,num_s,h]:
                                    action[num_p,m,arr,num_s,h] = a
                                    tmpactn[num_p,m,arr,num_s,h] = state_value
        
        print("iteration=",iteration)
        for p in all_pos:
            for m in range(0,max_mat):
                for arr in range(0,max_arrow):
                    for s in all_states:
                        for h in range(1,health):
                                print(end="(")
                                print(p,end=",")
                                print(m,end=",")
                                print(arr,end=",")
                                print(s,end=",")
                                print(h,end="):")
                                num_p = pos_map[p]
                                num_s = state_map[s]
                                print(action[num_p,m,arr,num_s,h],end="=[")
                                print(round(value[num_p,m,arr,num_s,h],3),end="]\n")

       #See if the loop should stop now         
        if biggest_change < DELTA:
            break
        iteration += 1

initial = sys.stdout
os.mkdir("./outputs")        

# task 1        
sys.stdout = open('./outputs/part_2_trace.txt', 'w')
iterate()
sys.stdout = initial

# task 2, case 1
E_left = 1
sys.stdout = open('./outputs/part_2_task_2.1_trace.txt', 'w')
# iterate()
sys.stdout = initial
E_left = 0

# task 2, case 2
action_reward['STAY'] = 0
sys.stdout = open('./outputs/part_2_task_2.2_trace.txt', 'w')
# iterate()
sys.stdout = initial
action_reward['STAY'] = -10

# task 2, case 3
GAMMA = 0.25
sys.stdout = open('./outputs/part_2_task_2.2_trace.txt', 'w')
# iterate()
sys.stdout = initial
GAMMA = 0.999

iteration= 0
(W,0,0,D,1):NONE=[0.0]
(W,0,0,D,2):NONE=[0.0]
(W,0,0,D,3):NONE=[0.0]
(W,0,0,R,1):NONE=[0.0]
(W,0,0,R,2):NONE=[0.0]
(W,0,0,R,3):NONE=[0.0]
(W,0,1,D,1):SHOOT=[2.5]
(W,0,1,D,2):NONE=[0.0]
(W,0,1,D,3):NONE=[0.0]
(W,0,1,R,1):SHOOT=[2.5]
(W,0,1,R,2):NONE=[0.0]
(W,0,1,R,3):NONE=[0.0]
(W,0,2,D,1):SHOOT=[2.5]
(W,0,2,D,2):NONE=[0.0]
(W,0,2,D,3):NONE=[0.0]
(W,0,2,R,1):SHOOT=[2.5]
(W,0,2,R,2):NONE=[0.0]
(W,0,2,R,3):NONE=[0.0]
(W,1,0,D,1):NONE=[0.0]
(W,1,0,D,2):NONE=[0.0]
(W,1,0,D,3):NONE=[0.0]
(W,1,0,R,1):NONE=[0.0]
(W,1,0,R,2):NONE=[0.0]
(W,1,0,R,3):NONE=[0.0]
(W,1,1,D,1):SHOOT=[2.5]
(W,1,1,D,2):NONE=[0.0]
(W,1,1,D,3):NONE=[0.0]
(W,1,1,R,1):SHOOT=[2.5]
(W,1,1,R,2):NONE=[0.0]
(W,1,1,R,3):NONE=[0.0]
(W,1,2,D,1):SHOOT=[2.5]
(W,1,2,D,2):NONE=[0.0]
(W,1,2,D,3):NONE=[0.0]
(W,1,2,R,1):SHOOT=[2.5]
(W,1,2,R,2):NONE=[0.0]
(W,1,2,R,3):NONE=[0.0]
(N,0,0,D,1):NONE=[0.0]
(N,0,0,D,2):NONE=[0.0]
(N,0,0,D,3):NONE=[0.0]
(N,0,0,R,1):NONE=[0.0]
(N,0,0,R,2):NONE=[0.0]
(N,0,0,R,3):NONE=[0.0]
(N,0,1,D,1):S

iteration= 2
(W,0,0,D,1):NONE=[0.0]
(W,0,0,D,2):NONE=[0.0]
(W,0,0,D,3):NONE=[0.0]
(W,0,0,R,1):NONE=[0.0]
(W,0,0,R,2):NONE=[0.0]
(W,0,0,R,3):NONE=[0.0]
(W,0,1,D,1):SHOOT=[2.5]
(W,0,1,D,2):NONE=[0.0]
(W,0,1,D,3):NONE=[0.0]
(W,0,1,R,1):SHOOT=[2.5]
(W,0,1,R,2):NONE=[0.0]
(W,0,1,R,3):NONE=[0.0]
(W,0,2,D,1):SHOOT=[2.969]
(W,0,2,D,2):NONE=[0.0]
(W,0,2,D,3):NONE=[0.0]
(W,0,2,R,1):SHOOT=[2.969]
(W,0,2,R,2):NONE=[0.0]
(W,0,2,R,3):NONE=[0.0]
(W,1,0,D,1):NONE=[0.0]
(W,1,0,D,2):NONE=[0.0]
(W,1,0,D,3):NONE=[0.0]
(W,1,0,R,1):NONE=[0.0]
(W,1,0,R,2):NONE=[0.0]
(W,1,0,R,3):NONE=[0.0]
(W,1,1,D,1):SHOOT=[2.5]
(W,1,1,D,2):NONE=[0.0]
(W,1,1,D,3):NONE=[0.0]
(W,1,1,R,1):SHOOT=[2.5]
(W,1,1,R,2):NONE=[0.0]
(W,1,1,R,3):NONE=[0.0]
(W,1,2,D,1):SHOOT=[2.969]
(W,1,2,D,2):NONE=[0.0]
(W,1,2,D,3):NONE=[0.0]
(W,1,2,R,1):SHOOT=[2.969]
(W,1,2,R,2):NONE=[0.0]
(W,1,2,R,3):NONE=[0.0]
(N,0,0,D,1):NONE=[0.0]
(N,0,0,D,2):NONE=[0.0]
(N,0,0,D,3):NONE=[0.0]
(N,0,0,R,1):NONE=[0.0]
(N,0,0,R,2):NONE=[0.0]
(N,0,0,R,3):NONE=[0.0]
(N,0,

iteration= 4
(W,0,0,D,1):NONE=[0.0]
(W,0,0,D,2):NONE=[0.0]
(W,0,0,D,3):NONE=[0.0]
(W,0,0,R,1):NONE=[0.0]
(W,0,0,R,2):NONE=[0.0]
(W,0,0,R,3):NONE=[0.0]
(W,0,1,D,1):SHOOT=[2.5]
(W,0,1,D,2):NONE=[0.0]
(W,0,1,D,3):NONE=[0.0]
(W,0,1,R,1):SHOOT=[2.5]
(W,0,1,R,2):NONE=[0.0]
(W,0,1,R,3):NONE=[0.0]
(W,0,2,D,1):SHOOT=[2.969]
(W,0,2,D,2):NONE=[0.0]
(W,0,2,D,3):NONE=[0.0]
(W,0,2,R,1):SHOOT=[2.969]
(W,0,2,R,2):NONE=[0.0]
(W,0,2,R,3):NONE=[0.0]
(W,1,0,D,1):NONE=[0.0]
(W,1,0,D,2):NONE=[0.0]
(W,1,0,D,3):NONE=[0.0]
(W,1,0,R,1):NONE=[0.0]
(W,1,0,R,2):NONE=[0.0]
(W,1,0,R,3):NONE=[0.0]
(W,1,1,D,1):SHOOT=[2.5]
(W,1,1,D,2):NONE=[0.0]
(W,1,1,D,3):NONE=[0.0]
(W,1,1,R,1):SHOOT=[2.5]
(W,1,1,R,2):NONE=[0.0]
(W,1,1,R,3):NONE=[0.0]
(W,1,2,D,1):SHOOT=[2.969]
(W,1,2,D,2):NONE=[0.0]
(W,1,2,D,3):NONE=[0.0]
(W,1,2,R,1):SHOOT=[2.969]
(W,1,2,R,2):NONE=[0.0]
(W,1,2,R,3):NONE=[0.0]
(N,0,0,D,1):NONE=[0.0]
(N,0,0,D,2):NONE=[0.0]
(N,0,0,D,3):NONE=[0.0]
(N,0,0,R,1):NONE=[0.0]
(N,0,0,R,2):NONE=[0.0]
(N,0,0,R,3):NONE=[0.0]
(N,0,

iteration= 6
(W,0,0,D,1):NONE=[0.0]
(W,0,0,D,2):NONE=[0.0]
(W,0,0,D,3):NONE=[0.0]
(W,0,0,R,1):NONE=[0.0]
(W,0,0,R,2):NONE=[0.0]
(W,0,0,R,3):NONE=[0.0]
(W,0,1,D,1):SHOOT=[2.5]
(W,0,1,D,2):NONE=[0.0]
(W,0,1,D,3):NONE=[0.0]
(W,0,1,R,1):SHOOT=[2.5]
(W,0,1,R,2):NONE=[0.0]
(W,0,1,R,3):NONE=[0.0]
(W,0,2,D,1):SHOOT=[2.969]
(W,0,2,D,2):NONE=[0.0]
(W,0,2,D,3):NONE=[0.0]
(W,0,2,R,1):SHOOT=[2.969]
(W,0,2,R,2):NONE=[0.0]
(W,0,2,R,3):NONE=[0.0]
(W,1,0,D,1):NONE=[0.0]
(W,1,0,D,2):NONE=[0.0]
(W,1,0,D,3):NONE=[0.0]
(W,1,0,R,1):NONE=[0.0]
(W,1,0,R,2):NONE=[0.0]
(W,1,0,R,3):NONE=[0.0]
(W,1,1,D,1):SHOOT=[2.5]
(W,1,1,D,2):NONE=[0.0]
(W,1,1,D,3):NONE=[0.0]
(W,1,1,R,1):SHOOT=[2.5]
(W,1,1,R,2):NONE=[0.0]
(W,1,1,R,3):NONE=[0.0]
(W,1,2,D,1):SHOOT=[2.969]
(W,1,2,D,2):NONE=[0.0]
(W,1,2,D,3):NONE=[0.0]
(W,1,2,R,1):SHOOT=[2.969]
(W,1,2,R,2):NONE=[0.0]
(W,1,2,R,3):NONE=[0.0]
(N,0,0,D,1):NONE=[0.0]
(N,0,0,D,2):NONE=[0.0]
(N,0,0,D,3):NONE=[0.0]
(N,0,0,R,1):NONE=[0.0]
(N,0,0,R,2):NONE=[0.0]
(N,0,0,R,3):NONE=[0.0]
(N,0,

iteration= 8
(W,0,0,D,1):NONE=[0.0]
(W,0,0,D,2):NONE=[0.0]
(W,0,0,D,3):NONE=[0.0]
(W,0,0,R,1):NONE=[0.0]
(W,0,0,R,2):NONE=[0.0]
(W,0,0,R,3):NONE=[0.0]
(W,0,1,D,1):SHOOT=[2.5]
(W,0,1,D,2):NONE=[0.0]
(W,0,1,D,3):NONE=[0.0]
(W,0,1,R,1):SHOOT=[2.5]
(W,0,1,R,2):NONE=[0.0]
(W,0,1,R,3):NONE=[0.0]
(W,0,2,D,1):SHOOT=[2.969]
(W,0,2,D,2):NONE=[0.0]
(W,0,2,D,3):NONE=[0.0]
(W,0,2,R,1):SHOOT=[2.969]
(W,0,2,R,2):NONE=[0.0]
(W,0,2,R,3):NONE=[0.0]
(W,1,0,D,1):NONE=[0.0]
(W,1,0,D,2):NONE=[0.0]
(W,1,0,D,3):NONE=[0.0]
(W,1,0,R,1):NONE=[0.0]
(W,1,0,R,2):NONE=[0.0]
(W,1,0,R,3):NONE=[0.0]
(W,1,1,D,1):SHOOT=[2.5]
(W,1,1,D,2):NONE=[0.0]
(W,1,1,D,3):NONE=[0.0]
(W,1,1,R,1):SHOOT=[2.5]
(W,1,1,R,2):NONE=[0.0]
(W,1,1,R,3):NONE=[0.0]
(W,1,2,D,1):SHOOT=[2.969]
(W,1,2,D,2):NONE=[0.0]
(W,1,2,D,3):NONE=[0.0]
(W,1,2,R,1):SHOOT=[2.969]
(W,1,2,R,2):NONE=[0.0]
(W,1,2,R,3):NONE=[0.0]
(N,0,0,D,1):NONE=[0.0]
(N,0,0,D,2):NONE=[0.0]
(N,0,0,D,3):NONE=[0.0]
(N,0,0,R,1):NONE=[0.0]
(N,0,0,R,2):NONE=[0.0]
(N,0,0,R,3):NONE=[0.0]
(N,0,